In [53]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

In [54]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # grok_api_key='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'
    # other params...
)

In [70]:

response= llm.invoke("can you cook food?")
print(response.content)

SyntaxError: invalid syntax (3600014593.py, line 1)

In [61]:
from langchain_core.prompts import PromptTemplate

def extract_job_and_contact_info(job_paragraph: str) -> str:
    prompt_extract = PromptTemplate.from_template("""
### JOB POSTING ANALYSIS TASK
Analyze this job posting and extract structured information:

{job_paragraph}

### OUTPUT REQUIREMENTS:
- Return JSON with two root keys: "employer_info" and "position_details"
- Output ONLY valid JSON - no commentary
- Include fields ONLY when explicitly mentioned
- Maintain original wording for values
- Handle all job types (medical, tech, education, etc.)

1. EMPLOYER_INFO (hiring organization/individual):
{{
    "full_name": "(if individual)",
    "organization": "(company/institution name)",
    "department": "(specific division/team)",
    "industry": "(medical, tech, education, etc.)",
    "contact": {{
        "email": "(preferred)",
        "phone": "(if provided)",
        "website": "(career portal/LinkedIn)"
    }},
    "location": {{
        "city": "(primary workplace)",
        "country": "(if mentioned)",
        "remote_options": "(hybrid/remote flags)"
    }},
    "organization_type": "(hospital, startup, university, etc.)",
    "key_attributes": ["list", "of", "notable", "features"]
}}

2. POSITION_DETAILS (job characteristics):
{{
    "title": "(official job name)",
    "type": "(full-time, contract, internship)",
    "category": "(clinical, engineering, research, etc.)",
    "level": "(junior, senior, principal)",
    "salary": {{
        "range": "(numbers or description)",
        "currency": "(if specified)",
        "bonuses": "(signing/performance bonuses)"
    }},
    "requirements": {{
        "education": "(degrees/certifications)",
        "experience": "(years/type)",
        "skills": ["technical", "and", "soft", "skills"],
        "licenses": "(industry-specific certifications)"
    }},
    "responsibilities": ["list", "of", "core", "duties"],
    "benefits": ["healthcare", "retirement", "perks"],
    "deadlines": {{
        "application": "(DD/MM/YYYY or relative)",
        "start_date": "(if specified)"
    }},
    "travel_requirements": "(percentage or description)",
    "reporting_structure": "(supervisory relationships)",
    "performance_metrics": "(KPIs/success measures)"
}}

### OUTPUT ONLY VALID JSON WITH NO MARKDOWN FORMATTING
""")
    chain_extract = prompt_extract | llm
    # prompt = prompt_extract.format(job_paragraph=job_paragraph)
    response = chain_extract.invoke({'job_paragraph': job_paragraph})
    return response.content

In [62]:
import re

def validate_or_ask_email(extracted_json: dict) -> str:
    # Check nested contact info first, then fallback to manual input
    email = extracted_json.get("employer_info", {}).get("contact", {}).get("email")
    
    if email and re.match(r"^[\w\.-]+@[a-zA-Z\d\.-]+\.[a-zA-Z]{2,}$", email):
        return email
    
    # If no valid email found, prompt user
    print("\n✉️ No valid email found in job posting. Please provide one:")
    while True:
        manual_email = input("Contact email: ").strip()
        if re.match(r"^[\w\.-]+@[a-zA-Z\d\.-]+\.[a-zA-Z]{2,}$", manual_email):
            return manual_email
        print("Invalid format. Please enter a valid email address (e.g. name@company.com)")

In [67]:
from langchain_core.output_parsers import JsonOutputParser
def run_outreach_intake():
    print("📋 Paste the job posting below:")
    job_text = input("Job Posting: ")

    print("\n🔍 Extracting details using LLM...")
    ext = extract_job_and_contact_info(job_text)
    parser = JsonOutputParser()
    extracted = parser.parse(ext)
    print("\n✅ Extracted JSON:\n", extracted)

    extracted_json = extracted

    # Get nested contact information
    contact_info = extracted_json.get("employer_info", {}).get("contact", {})
    contact_email = validate_or_ask_email({
        "contact_email": contact_info.get("email"),
        **extracted_json
    })

    print("\n📌 Final Info:")
    print(f"Industry: {extracted_json.get('employer_info', {}).get('industry', 'N/A')}")
    print(f"Organization: {extracted_json.get('employer_info', {}).get('organization', 'N/A')}")
    print("Contact Email:", contact_email)
    print("\nPosition Details:")
    for key, value in extracted_json.get("position_details", {}).items():
        print(f"- {key.replace('_', ' ').title()}: {value or 'N/A'}")

    return {
        "employer_info": extracted_json.get("employer_info", {}),
        "position_details": extracted_json.get("position_details", {}),
        "contact_email": contact_email
    }

In [68]:
run_outreach_intake()

📋 Paste the job posting below:


Job Posting:  𝐅𝐢𝐧𝐚𝐥 𝐂𝐚𝐥𝐥  𝐎𝐧𝐥𝐲 𝟑 𝐃𝐚𝐲𝐬 𝐋𝐞𝐟𝐭  𝑨𝒑𝒑𝒍𝒊𝒄𝒂𝒕𝒊𝒐𝒏𝒔 𝒇𝒐𝒓 𝒕𝒉𝒆 𝑨𝑰/𝑴𝑳 𝑰𝒏𝒕𝒆𝒓𝒏𝒔𝒉𝒊𝒑 𝑷𝒓𝒐𝒈𝒓𝒂𝒎 𝒂𝒕 𝑨𝒓𝒕𝒊𝒇𝒊𝒄𝒊𝒛𝒆𝒏 𝒂𝒓𝒆 𝒄𝒍𝒐𝒔𝒊𝒏𝒈 𝒔𝒐𝒐𝒏!  If you've been planning to apply, now's the time. No further extensions.  Don’t miss the opportunity to join one of the most exciting AI Internship Program.  𝐃𝐞𝐚𝐝𝐥𝐢𝐧𝐞: 𝟏𝟎-𝟎𝟓-𝟐𝟎𝟐𝟓  𝐀𝐩𝐩𝐥𝐲 𝐚𝐭: hiring@artificizen.com



🔍 Extracting details using LLM...

✅ Extracted JSON:
 {'employer_info': {'organization': 'Artificizen', 'industry': 'Tech', 'contact': {'email': 'hiring@artificizen.com'}, 'location': {'city': 'Not specified'}, 'organization_type': 'Startup', 'key_attributes': []}, 'position_details': {'title': 'AI Internship Program', 'type': 'Internship', 'category': 'Research', 'level': 'Junior', 'salary': {'range': 'Not specified'}, 'requirements': {'education': 'Not specified', 'experience': 'Not specified', 'skills': [], 'licenses': 'Not specified'}, 'responsibilities': ['Join one of the most exciting AI Internship Program'], 'benefits': [], 'deadlines': {'application': '10-05-2025'}, 'travel_requirements': 'Not specified', 'reporting_structure': 'Not specified', 'performance_metrics': 'Not specified'}}

📌 Final Info:
Industry: Tech
Organization: Artificizen
Contact Email: hiring@artificizen.com

Position Details:
- Title: AI Internship Program
- Type: Internship
- Category: Research
- Level: Ju

{'employer_info': {'organization': 'Artificizen',
  'industry': 'Tech',
  'contact': {'email': 'hiring@artificizen.com'},
  'location': {'city': 'Not specified'},
  'organization_type': 'Startup',
  'key_attributes': []},
 'position_details': {'title': 'AI Internship Program',
  'type': 'Internship',
  'category': 'Research',
  'level': 'Junior',
  'salary': {'range': 'Not specified'},
  'requirements': {'education': 'Not specified',
   'experience': 'Not specified',
   'skills': [],
   'licenses': 'Not specified'},
  'responsibilities': ['Join one of the most exciting AI Internship Program'],
  'benefits': [],
  'deadlines': {'application': '10-05-2025'},
  'travel_requirements': 'Not specified',
  'reporting_structure': 'Not specified',
  'performance_metrics': 'Not specified'},
 'contact_email': 'hiring@artificizen.com'}